<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 22s 840ms/step - loss: 0.6655 - accuracy: 0.5864 - val_loss: 0.6555 - val_accuracy: 0.5463
Epoch 2/10
26/26 [==============================] - 20s 784ms/step - loss: 0.6215 - accuracy: 0.6800 - val_loss: 0.5997 - val_accuracy: 0.7659
Epoch 3/10
26/26 [==============================] - 20s 769ms/step - loss: 0.5668 - accuracy: 0.7737 - val_loss: 0.6970 - val_accuracy: 0.4634
Epoch 4/10
26/26 [==============================] - 20s 777ms/step - loss: 0.5127 - accuracy: 0.7616 - val_loss: 0.4458 - val_accuracy: 0.8878
Epoch 5/10
26/26 [==============================] - 22s 831ms/step - loss: 0.4354 - accuracy: 0.8309 - val_loss: 0.4428 - val_accuracy: 0.7756
Epoch 6/10
26/26 [==============================] - 21s 801ms/step - loss: 0.3888 - accuracy: 0.8504 - val_loss: 0.3648 - val_accuracy: 0.8927
Epoch 7/10
26/26 [==============================] - 20s 781ms/step - loss: 0.3280 - accuracy: 0.8796 - val_loss: 0.2778 - val_accuracy: 0.9220

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.67.h5
26/26 - 22s - loss: 0.6754 - accuracy: 0.5633 - val_loss: 0.6677 - val_accuracy: 0.4780
Epoch 2/5

Epoch 00002: saving model to weights.02-0.60.h5
26/26 - 20s - loss: 0.6290 - accuracy: 0.7007 - val_loss: 0.5982 - val_accuracy: 0.7610
Epoch 3/5

Epoch 00003: saving model to weights.03-0.53.h5
26/26 - 22s - loss: 0.5587 - accuracy: 0.7701 - val_loss: 0.5298 - val_accuracy: 0.7854
Epoch 4/5

Epoch 00004: saving model to weights.04-0.50.h5
26/26 - 20s - loss: 0.5107 - accuracy: 0.7774 - val_loss: 0.4976 - val_accuracy: 0.7902
Epoch 5/5

Epoch 00005: saving model to weights.05-0.45.h5
26/26 - 20s - loss: 0.4591 - accuracy: 0.7859 - val_loss: 0.4526 - val_accuracy: 0.8098


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 25s - loss: 0.6718 - accuracy: 0.6095 - val_loss: 0.6584 - val_accuracy: 0.6341


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 19s - loss: 0.6496 - accuracy: 0.6265 - val_loss: 0.6129 - val_accuracy: 0.6390
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 19s - loss: 0.6032 - accuracy: 0.6837 - val_loss: 0.6030 - val_accuracy: 0.6195


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6816 - accuracy: 0.5438 - val_loss: 0.6764 - val_accuracy: 0.5171
Epoch 2/50
26/26 - 19s - loss: 0.6523 - accuracy: 0.6363 - val_loss: 0.6693 - val_accuracy: 0.4927
Epoch 3/50
26/26 - 19s - loss: 0.6127 - accuracy: 0.7202 - val_loss: 0.5968 - val_accuracy: 0.7220
Epoch 4/50
26/26 - 19s - loss: 0.5536 - accuracy: 0.7822 - val_loss: 0.5641 - val_accuracy: 0.6927
Epoch 5/50
26/26 - 19s - loss: 0.4961 - accuracy: 0.7895 - val_loss: 0.4472 - val_accuracy: 0.8098
Epoch 6/50
26/26 - 19s - loss: 0.4510 - accuracy: 0.8066 - val_loss: 0.4033 - val_accuracy: 0.8683
Epoch 7/50
26/26 - 18s - loss: 0.3773 - accuracy: 0.8528 - val_loss: 0.3218 - val_accuracy: 0.9024
Epoch 8/50
26/26 - 18s - loss: 0.3256 - accuracy: 0.8881 - val_loss: 0.2379 - val_accuracy: 0.9512
Epoch 9/50
26/26 - 18s - loss: 0.2302 - accuracy: 0.9355 - val_loss: 0.1768 - val_accuracy: 0.9805
Epoch 10/50
26/26 - 18s - loss: 0.1991 - accuracy: 0.9453 - val_loss: 0.1529 - val_accuracy: 0.9805
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 19s 721ms/step - loss: 0.6783 - accuracy: 0.5839 - val_loss: 0.6856 - val_accuracy: 0.4390
Epoch 2/5
26/26 [==============================] - 18s 700ms/step - loss: 0.6445 - accuracy: 0.6411 - val_loss: 0.6551 - val_accuracy: 0.5268
Epoch 3/5
26/26 [==============================] - 18s 708ms/step - loss: 0.6065 - accuracy: 0.7044 - val_loss: 0.5709 - val_accuracy: 0.8146
Epoch 4/5
26/26 [==============================] - 18s 700ms/step - loss: 0.5555 - accuracy: 0.7311 - val_loss: 0.5223 - val_accuracy: 0.8146
Epoch 5/5
26/26 [==============================] - 18s 692ms/step - loss: 0.5111 - accuracy: 0.7689 - val_loss: 0.4592 - val_accuracy: 0.8439


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.583942,0.678476,0.439024,0.685583
1,1,0.641119,0.644677,0.526829,0.655062
2,2,0.704380,0.606691,0.814634,0.570920
3,3,0.731144,0.556815,0.814634,0.522279
4,4,0.768856,0.512973,0.843902,0.459243


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 29s 1s/step - loss: 0.6714 - accuracy: 0.6290 - val_loss: 0.6823 - val_accuracy: 0.4488

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 19s 722ms/step - loss: 0.6439 - accuracy: 0.6277 - val_loss: 0.6449 - val_accuracy: 0.7610

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 19s 716ms/step - loss: 0.6293 - accuracy: 0.7518 - val_loss: 0.6358 - val_accuracy: 0.7805

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 19s 716ms/step - loss: 0.6205 - accuracy: 0.7920 - val_loss: 0.6327 - val_accuracy: 0.7659

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 18s 711ms/step - loss: 0.6165

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 20s 775ms/step - loss: 0.6820 - accuracy: 0.5596 - val_loss: 0.7284 - val_accuracy: 0.4341
Epoch 2/50
26/26 [==============================] - 19s 729ms/step - loss: 0.6271 - accuracy: 0.6606 - val_loss: 0.5837 - val_accuracy: 0.8634
Epoch 3/50
26/26 [==============================] - 19s 715ms/step - loss: 0.5753 - accuracy: 0.7457 - val_loss: 0.5685 - val_accuracy: 0.6732
Epoch 4/50
26/26 [==============================] - 18s 711ms/step - loss: 0.5079 - accuracy: 0.7871 - val_loss: 0.4375 - val_accuracy: 0.8829
Epoch 5/50
26/26 [==============================] - 19s 731ms/step - loss: 0.4288 - accuracy: 0.8321 - val_loss: 0.3944 - val_accuracy: 0.8537
Epoch 6/50
26/26 [==============================] - 18s 708ms/step - loss: 0.3653 - accuracy: 0.8662 - val_loss: 0.2853 - val_accuracy: 0.9366
Epoch 7/50
25/26 [===========================>..] - ETA: 0s - loss: 0.2893 - accuracy: 0.9150
Epoch 00007: ReduceLROnPlateau reducing learning

25/26 [===========================>..] - ETA: 0s - loss: 0.0808 - accuracy: 0.9800
Epoch 00044: ReduceLROnPlateau reducing learning rate to 0.001.
26/26 [==============================] - 19s 738ms/step - loss: 0.0803 - accuracy: 0.9805 - val_loss: 0.0802 - val_accuracy: 0.9756
Epoch 45/50
25/26 [===========================>..] - ETA: 0s - loss: 0.0779 - accuracy: 0.9850
Epoch 00045: ReduceLROnPlateau reducing learning rate to 0.001.
26/26 [==============================] - 19s 727ms/step - loss: 0.0778 - accuracy: 0.9854 - val_loss: 0.0788 - val_accuracy: 0.9756
Epoch 46/50
25/26 [===========================>..] - ETA: 0s - loss: 0.0771 - accuracy: 0.9837
Epoch 00046: ReduceLROnPlateau reducing learning rate to 0.001.
26/26 [==============================] - 19s 735ms/step - loss: 0.0766 - accuracy: 0.9842 - val_loss: 0.0789 - val_accuracy: 0.9756
Epoch 47/50
26/26 [==============================] - 19s 731ms/step - loss: 0.0753 - accuracy: 0.9842 - val_loss: 0.0770 - val_accuracy: 0.

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 5054), started 0:15:47 ago. (Use '!kill 5054' to kill it.)